# 函数式编程

在Python中, 一切皆对象, 函数当然也是一个对象.这里我们可以将一个函数, 作为一个对象传入, 实现一些高阶的功能.

# map 

map的功能就是就列表的每一个元素都进行function的操作

In [1]:
def add(x):
    return x**2
l = [1,2,3,4,5,6,7]
print(list(map(add, l)))# 使用list将其变成一个列表, 否则还是一个迭代器

[1, 4, 9, 16, 25, 36, 49]


# reduce

reduce的功能与map类似, 它是作用于需要有两个输入参数的函数上面, 其操作结果为

l = [x,y,z,a]

reduce(f, l) --->  f(f(f(x,y),z),a)

In [2]:
from functools import reduce
def func(x, y):
    return x*10+y
l = [1,2,3,4,5,6,7]
print(reduce(func, l))

1234567


类似的, 我们可以自己设计一个将整型字符变成整型数字的函数

In [3]:
str2intset = {'0':0, '1':1, '2':2, '3':3,'4':4, '5':5,'6':6,'7':7,'8':8,'9':9}
def str2int(a):
    return str2intset[a]
def func(x,y):
    return 10*x+y
a = '1234567'
reduce(func, list(map(str2int, a)))

1234567

我们也可以将一个浮点数字符串变成一个浮点数数字

In [4]:
str2intset = {'0':0, '1':1, '2':2, '3':3,'4':4, '5':5,'6':6,'7':7,'8':8,'9':9}
def str2int(a):
    return str2intset[a]
def func(x,y):
    return 10*x+y
a = '1234.567'
a1, a2 = a.split('.')
n = len(a2)
a1 = reduce(func, list(map(str2int, a1)))
a2 = reduce(func, list(map(str2int, a2)))
print(a1+a2*10**(-n))

1234.567


# filter

filter的功能是筛选, 如果函数返回True则留下, False就会被筛选掉

In [5]:
def func(x):
    return x%2 == 0
l = [1,2,3,4,5,6,7]
print(list(filter(func, l)))

[2, 4, 6]


下面用埃氏筛法来寻找质数, 其中会用到filter函数

In [6]:
def odd():
    n = 3
    while True:
        yield n
        n += 2
def is_division(i):
    return lambda x:x%i>0

def prime():
    yield 2
    it = odd()
    while True:
        n = next(it)
        yield n
        it = filter(is_division(n), it)

l = []
for i in prime():
    if i < 100:
        l.append(i)
    else:
        break
print(l)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


# sorted

这是Python内置的排序函数, 可以对一个列表进行排序

In [7]:
l = [3,2,4,6,1,5,7,5,9]
l1 = sorted(l)
print('l:', l)
print('l1:', l1)# sorted排序不会改变原有的列表的次序

l: [3, 2, 4, 6, 1, 5, 7, 5, 9]
l1: [1, 2, 3, 4, 5, 5, 6, 7, 9]


这里我们还可以使用key, 使得元素可以按照某种顺序进行排序

In [8]:
l = [1,2,-4,5,-9,10,4,5,2,-6,8]
sorted(l,key=abs) # 按照绝对值大小进行排序

[1, 2, 2, -4, 4, 5, 5, -6, 8, -9, 10]

In [9]:
l = ['bob', 'about', 'Zoo', 'Credit']
sorted(l, key=str.lower)# 按照字母数字的ascii码进行排序

['about', 'bob', 'Credit', 'Zoo']

In [10]:
def by_name(t):
    return t[0].lower()
L = [('Bob', 75), ('Adam', 92), ('Bart', 66), ('Lisa', 88)]
sorted(L,key=by_name)

[('Adam', 92), ('Bart', 66), ('Bob', 75), ('Lisa', 88)]

# 返回函数

由于Python的机制决定, 我们可以将函数作为返回值, 例如下面的例子

In [11]:
def lazy_sum(a, b):
    def sum_():
        return a+b
    return sum_

f1 = lazy_sum(3,4)
f2 = lazy_sum(5,6)
print(f1())
print(f2())

7
11


需要注意的问题是，返回的函数并没有立刻执行，而是直到调用了f()才执行。我们来看一个例子：

In [12]:
def count():
    fs = []
    for i in range(1, 4):
        def f():
             return i*i
        fs.append(f)
    return fs

f1, f2, f3 = count()
print([f1(), f2(), f3()]) 

[9, 9, 9]


输出并不是[1, 4, 9]而是[9, 9, 9]这就是由于闭包并不是立即执行的, 而是直到调用才开始执行, 所以当i = 3时才开始执行, 所以导致最终结果都是9

如果想得到预期的结果, 那么就应该再定义一个函数, 最后执行的时候才可以将数值赋值进去.

In [13]:
def count():
    def f(i):
        def g():
            return i**2
        return g
    l = []
    for i in range(1,4):
        l.append(f(i))
    return l
f1,f2,f3 = count()
print([f1(), f2(), f3()])

[1, 4, 9]


In [14]:
def createCounter():
    count = [0]
    def counter():
        count[0] += 1
        return count[0]
    return counter

counterA = createCounter()
print(counterA(), counterA(), counterA(), counterA(), counterA()) # 1 2 3 4 5
counterB = createCounter()
if [counterB(), counterB(), counterB(), counterB()] == [1, 2, 3, 4]:
    print('测试通过!')
else:
    print('测试失败!')

1 2 3 4 5
测试通过!


In [15]:
def createCounter():
    count = 0
    def counter():
        nonlocal count 
        count += 1
        return count
    return counter

counterA = createCounter()
print(counterA(), counterA(), counterA(), counterA(), counterA()) # 1 2 3 4 5
counterB = createCounter()
if [counterB(), counterB(), counterB(), counterB()] == [1, 2, 3, 4]:
    print('测试通过!')
else:
    print('测试失败!')

1 2 3 4 5
测试通过!


In [16]:
# 生成器解法
def createCounter():
    def f():
        n = 1
        while True:
            yield n
            n += 1
    g = f()
    def counter():
        return next(g)
    return counter
counterA = createCounter()
print(counterA(), counterA(), counterA(), counterA(), counterA()) # 1 2 3 4 5
counterB = createCounter()
if [counterB(), counterB(), counterB(), counterB()] == [1, 2, 3, 4]:
    print('测试通过!')
else:
    print('测试失败!')

1 2 3 4 5
测试通过!


# 变量作用域

这里我们讲一下Python中的变量作用域的问题




















python引用变量的顺序： 当前作用域局部变量->外层作用域变量->当前模块中的全局变量->python内置变量.

global关键字用来在函数或其他局部作用域中使用全局变量。但是如果不修改全局变量也可以不使用global关键字

In [17]:
gcount = 0
def global_test():
    gcount+=1
    print (gcount)
print([global_test(), global_test(), global_test()])

UnboundLocalError: local variable 'gcount' referenced before assignment

第一行定义了一个全局变量, (可以省略global关键字)

在global_test 函数中程序会因为“如果内部函数有引用外部函数的同名变量或者全局变量,并且对这个变量有修改.那么python会认为它是一个局部变量,又因为函数中没有gcount的定义和赋值，所以报错. 所以在使用之前需要先声明是全局变量.

In [18]:
gcount = 0
def global_test():
    global gcount
    gcount+=1
    return gcount
print(global_test(), global_test(), global_test())

1 2 3


如果局部没有声明全局变量, 但是只是使用全局变量, 没有修改这也是可以的.

In [19]:
gcount = 0
def global_test():
    return gcount
print(global_test(), global_test(), global_test())

0 0 0


# nonlocal

nonlocal 关键字用来在函数或其他作用域中使用外层(非全局)变量。

In [20]:
def make_counter(): 
    count = 0 
    def counter(): 
        nonlocal count 
        count += 1 
        return count 
    return counter 
def make_counter_test(): 
    mc = make_counter() 
    print(mc())
    print(mc())
    print(mc())
make_counter_test()

1
2
3


In [21]:
def scope_test():
    def do_local():
        spam = "local spam" #此函数定义了另外的一个spam字符串变量，并且生命周期只在此函数内。此处的spam和外层的spam是两个变量，如果写出spam = spam + “local spam” 会报错
    def do_nonlocal():
        nonlocal  spam        #使用外层的spam变量
        spam = "nonlocal spam"
    def do_global():
        global spam
        spam = "global spam"
    spam = "test spam"
    do_local()
    print("After local assignmane:", spam)#test spam
    do_nonlocal()
    print("After nonlocal assignment:",spam)#nonlocal spam
    do_global()
    print("After global assignment:",spam)#nonlocal spam
scope_test()
print("In global scope:",spam)#global spam

After local assignmane: test spam
After nonlocal assignment: nonlocal spam
After global assignment: nonlocal spam
In global scope: global spam


上面这个例子就可以看到, Python引用变量的顺序为: 首先查找当前作用域下的局部变量, 接着是nonlocal变量, 接着是当前模块的global全局变量, 最后才会查找Python内置的全局变量

下面再举几个例子

In [22]:
def add_b():
    global  b
    b = 42
    def do_global():
        global  b
        b = b + 10
        print(b)
    do_global()
    print(b)
add_b()

52
52


In [23]:
def add_b():
    global  b
    b = 42
    def do_global():
        global  a
        a = b + 10
        print(b)
    do_global()
    print(a)
add_b()
print("a = %s , b = %s " %(a, b))

42
52
a = 52 , b = 42 


In [24]:
def add_b():
    #global  b
    b = 42
    def do_global():
        global  b
        b =  10
        print(b)
    do_global()
    print(b)
add_b()
print(" b = %s " % b)

10
42
 b = 10 


In [25]:
def add_b():
    #global  b
    b = 42
    def do_global():
        nonlocal  b
        b =  10
        print(b)
    do_global()
    print(b)
add_b()

10
10
